In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import socket
import struct
import io
from PIL import Image
import webcolors

ex, ey, ex1, ey1, ex2, sx2, sy2, ey2, hy2, hy1, hy, hx1, hx2 = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

def closest_color_name(requested_color):
    min_color_diff = None
    closest_color_name = None

    for color_name, color_hex in webcolors.CSS3_NAMES_TO_HEX.items():
        color_rgb = webcolors.hex_to_rgb(color_hex)
        color_diff = sum(abs(c1 - c2) for c1, c2 in zip(requested_color, color_rgb))

        if min_color_diff is None or color_diff < min_color_diff:
            min_color_diff = color_diff
            closest_color_name = color_name

    return closest_color_name


def get_pixel_color(image, x, y):
    if isinstance(image, np.ndarray):
        pixel_color = image[y, x]
    else:
        pixel_color = image.getpixel((x, y))
    return pixel_color

def get_average_color(img, coords, width, height):
    region = img[coords[1]:coords[1] + height, coords[0]:coords[0] + width]
    return np.mean(region, axis=(0, 1)).astype(int)

def process_image(img):
    global ex, ey, ex1, ey1, ex2, sx2, sy2, ey2, hy2, hy1, hy, hx1, hx2,iw,lm,ecolor, hcolor, scolor
    ecolor, hcolor, scolor=0,0,0

    NUM_FACE = 2
    mpDraw = mp.solutions.drawing_utils
    mpFaceMesh = mp.solutions.face_mesh
    faceMesh = mpFaceMesh.FaceMesh(max_num_faces=NUM_FACE)
    drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=1)

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = faceMesh.process(imgRGB)

    if results.multi_face_landmarks:
        for faceLms in results.multi_face_landmarks:
            #mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACEMESH_TESSELATION, drawSpec, drawSpec)
            for id, lm in enumerate(faceLms.landmark):
                iw,ih,ecolor, hcolor, scolor=0,0,0,0,0
                if id == 386:
                    ih, iw, ic = img.shape
                    ex1, ey1 = int(lm.x * iw), int(lm.y * ih)
                if id == 374:
                    ih, iw, ic = img.shape
                    ex2, ey2 = int(lm.x * iw), int(lm.y * ih)
                ex = ex2 + ex1
                ey = ey2 + ey1

                if id == 426:
                    ih, iw, ic = img.shape
                    sx2, sy2 = int(lm.x * iw), int(lm.y * ih)
                if id == 10:
                    hx2, hy2 = int(lm.x * iw), int(lm.y * ih)
                if id == 168:
                    hx1, hy1 = int(lm.x * iw), int(lm.y * ih)
                hy = hy2 - hy1
                hy = hy2 + (hy + (hy * 0.2))

                eyecolor = get_pixel_color(imgRGB, int(ex / 2), int(ey / 2))
                skincolor = get_pixel_color(imgRGB, (sx2), (sy2))
                haircolor = get_pixel_color(imgRGB, (int(hx2)), (int(hy)))
                ecolor = closest_color_name(eyecolor)
                scolor = closest_color_name(skincolor)
                hcolor = closest_color_name(haircolor)
                

    return ecolor, hcolor, scolor

def main(image_file_path):
    img = cv2.imread(image_file_path)
    ecolor, hcolor, scolor = process_image(img)
    print(f"eye color={ecolor}, skin color={scolor}, hair color={hcolor}")

if __name__ == "__main__":
    image_file_path = "shah.jpg"
    main(image_file_path)